# Background Tasks

> Background tasks are functions run after returning a response.

Useful for operations where you want the user to get a response quickly but you don't want or need the user waiting for it to finish. Typical scenarios include:

- User setup in complex systems where you can inform the user later in email that their account is complete
- Batch processes that can take a significant amount of time (bulk email or API calls)
- Any other process where the user can be notified later by email, websocket, webhook, or pop-up

Background tasks are an easy-to-use wrapper over Python's concurrency mechanisms. They are used to improve user experience, often making apps feel faster to the end user.

## A simple background task example

In this example attaching a task to FtResponse by assigning it via the background argument. When the page is visited, it will display 'Simple Background Task Example' almost instantly, while in the terminal it will slowly count upward from 0.

``` {.python filename="main.py" code-line-numbers="true"}
from fasthtml.common import *
from starlette.background import BackgroundTask
from time import time, sleep

app, rt = fast_app()

def counter(loops:int): # <1>
    """Slowly print integers to the terminal"""
    for i in range(loops):
        print(i)
        sleep(i)

@rt
def index():
    task = BackgroundTask(counter, loops=5)  # <2>
    cts = Titled('Simple Background Task Example')
    return FtResponse(cts, background=task) # <3>

serve()
```

1. `counter` is our task function. There is nothing special about it, although it is a good practice for its arguments to be serializable as JSON
2. We use `starlette.background.BackgroundTask` to turn the counter function into a background task
3. `FtResponse` is called explicitly so we can attack the task to its background. Normally we don't need to call `FtResponse` explicitly, setting background tasks is the exception.

## A more realistic example

Let's imagine that we are accessing a slow-to-process critical service. We don't want our users to have to wait. While we could set up SSE to notify on completion, instead we decide to periodically check to see if the status of their record has changed.

So first, let's create a very simple slow timestamp API. What it does is stall requests for 10 seconds before returning JSON containing a timestamp.

In [ ]:
from fasthtml.common import *
from time import sleep, time

app, rt = fast_app()

@rt
def index():
    sleep(10)
    return {'timestamp': time()}

serve(port=8123)

Now let's create a website that uses this API to fetch the timestamp from the glacially slow service.

```python
from fasthtml.common import *
from fastlite import *
from starlette.background import BackgroundTask
from time import time, sleep
import httpx

app, rt = fast_app()

db = database('data.db')

class Moment: timestamp: float
moments = db.create(Moment, pk='timestamp')


async def submit_record():
    client = httpx.AsyncClient()
    r = await client.post('http://127.0.0.1:8123', timeout=20)
    moments.insert(timestamp=r.json()['timestamp'])

        
@rt('/submit-record')
async def post():
    cts = H3(f'Record submitted at {time()}, click again', hx_post='submit-record')
    task = BackgroundTask(submit_record)
    return FtResponse(cts, background=task)

@rt
def index():
    timestamps = moments(order_by='timestamp')
    return Titled('Slow access dashboard',
        H2('Click here to ping slow service', hx_post='/submit-record'),
        H3('Refresh the page to check the latest timestamp'),        
        P(timestamps),
    )

serve()
```

In [ ]:
%%ai
How to use HTMX to ping a route every 5 seconds

KeyError: 'nbmeta'

::: {.callout-caution}
## Background tasks are not distributed task queues

While background tasks often provides the user with a faster experience, the server itself isn't accelerated. What that means is that processes are still happening, but are hidden. So if a server is struggling under the load of a lot of user activity, so long as it isn't an issue with HTTP, background tasks won't help with server load.

This is where full-fledged distributed task queue libraries like Celery and Dramatiq come into play. At the cost of dramatically increased complexity over background tasks they allow for the distribution of tasks over addition servers, as well as providing improved observability, retry mechanisms, and persistence in case of server shutdown.

In our experience, it's often better to build something with background tasks and then convert it to a task queue.
:::